In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unidecode
from PyPDF2 import PdfReader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


 Bibliothèques importées


Configuration des chemins

In [2]:
#les fichiers à analyser
cv_files = ["1CV-barman.pdf", "1CV-vendeur.pdf", "1CV-boucher.pdf", "1cv-patissier.pdf"]
offre_files = ["1Offre-barman.pdf", "1Offre-vendeur.pdf", "1Offre-boucher.pdf", "1Offre-patissier.pdf"]

print(f" Répertoire : {os.getcwd()}")
print(f" {len(cv_files)} CV et {len(offre_files)} offres à traiter")

 Répertoire : c:\Users\dell\Documents\cours M2\projet_ing\Projet-MARS\test_nassima
 4 CV et 4 offres à traiter


Fonctions d'extraction et nettoyage

In [3]:
def clean_text(text):
    """Nettoie le texte (accents, espaces, caractères spéciaux)"""
    if not isinstance(text, str):
        return ""
    
    text = unidecode.unidecode(text)
    replacements = {"*": "", "'": "'", "'": "'", "\t": " ", "\n": " ", "\r": ""}
    for old, new in replacements.items():
        text = text.replace(old, new)
    
    return ' '.join(text.split()).strip()

def extract_text_from_pdf(pdf_path):
    """Extrait le texte d'un PDF en un bloc"""
    try:
        reader = PdfReader(pdf_path)
        pages = [page.extract_text() for page in reader.pages]
        full_text = "\n".join(pages)
        return clean_text(full_text)
    except Exception as e:
        print(f" Erreur {pdf_path}: {e}")
        return ""

print(" Fonctions définies")

 Fonctions définies


Extraction des CV (1 bloc)

In [4]:
cv_textes = []
cv_noms = []

for cv in cv_files:
    if os.path.exists(cv):
        texte = extract_text_from_pdf(cv)
        cv_textes.append(texte)
        cv_noms.append(cv.replace('.pdf', ''))
        print(f"✓ {cv}: {len(texte)} caractères")
    else:
        print(f" {cv} non trouvé")

print(f"\n {len(cv_textes)} CV extraits")

 1CV-barman.pdf non trouvé
 1CV-vendeur.pdf non trouvé
 1CV-boucher.pdf non trouvé
 1cv-patissier.pdf non trouvé

 0 CV extraits


Extraction des offres (1 bloc)

In [5]:
offre_textes = []
offre_noms = []

for offre in offre_files:
    if os.path.exists(offre):
        texte = extract_text_from_pdf(offre)
        offre_textes.append(texte)
        offre_noms.append(offre.replace('.pdf', ''))
        print(f"✓ {offre}: {len(texte)} caractères")
    else:
        print(f"  {offre} non trouvé")

print(f"\n {len(offre_textes)} offres extraites")


  1Offre-barman.pdf non trouvé
  1Offre-vendeur.pdf non trouvé
  1Offre-boucher.pdf non trouvé
  1Offre-patissier.pdf non trouvé

 0 offres extraites


Vectorisation TF-IDF

In [8]:
# Vérification avant vectorisation
if not cv_textes or not offre_textes:
    print(" ARRÊT : Pas de textes à vectoriser")
    print("Vérifiez l'extraction des PDF à l'étape précédente")
else:
    # Initialiser le vectoriseur
    vectorizer = TfidfVectorizer(
        max_features=100,
        ngram_range=(1, 2),
        min_df=1
    )
    
    try:
        # Vectoriser CV et offres
        X_cv = vectorizer.fit_transform(cv_textes)
        X_offres = vectorizer.transform(offre_textes)
        
        print(f" Matrice CV : {X_cv.shape}")
        print(f" Matrice Offres : {X_offres.shape}")
        print(f" Vocabulaire : {len(vectorizer.vocabulary_)} termes")
    except ValueError as e:
        print(f" ERREUR de vectorisation : {e}")
        print("\nSOLUTIONS POSSIBLES :")
        print("1. Les PDF sont des images (texte non extractible)")
        print("   → Utilisez pdfplumber ou un OCR (tesseract)")
        print("2. Les textes sont trop courts")
        print("   → Réduisez min_df ou vérifiez le contenu")
        print("\nContenu des textes :")
        for i, txt in enumerate(cv_textes[:2]):
            print(f"CV {i+1}: '{txt[:200]}'...")

 ARRÊT : Pas de textes à vectoriser
Vérifiez l'extraction des PDF à l'étape précédente


Calcul des similarités

In [10]:
# Vérifier que X_cv et X_offres existent
try:
    # Calcul de la similarité cosinus
    similarities = cosine_similarity(X_cv, X_offres)
    distances = 1 - similarities
    
    print("\n Matrice de similarité :")
    df_sim = pd.DataFrame(similarities, index=cv_noms, columns=offre_noms)
    print(df_sim.round(3))
except NameError:
    print(" ERREUR : X_cv ou X_offres non défini")
    print("La vectorisation TF-IDF (cellule précédente) a échoué.")
    print("\nRetournez à la cellule 6 et vérifiez les messages d'erreur.")


 ERREUR : X_cv ou X_offres non défini
La vectorisation TF-IDF (cellule précédente) a échoué.

Retournez à la cellule 6 et vérifiez les messages d'erreur.


Visualisation : Matrice de distances

In [11]:
plt.figure(figsize=(10, 8))
sns.heatmap(
    distances,
    annot=True,
    fmt=".2f",
    cmap="viridis",
    square=True,
    xticklabels=offre_noms,
    yticklabels=cv_noms,
    cbar_kws={'label': 'Distance'}
)
plt.title("Matrice de distances TF-IDF (1 - Similarité)", fontsize=14, fontweight='bold')
plt.xlabel("Offres")
plt.ylabel("CV")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

NameError: name 'distances' is not defined

<Figure size 1000x800 with 0 Axes>

Meilleurs matchs

In [ ]:
if similarities is not None:
    print("\n MEILLEURS MATCHS :")
    for i, cv in enumerate(cv_noms):
        best_idx = np.argmin(distances[i])
        print(f"{cv} → {offre_noms[best_idx]} ({similarities[i, best_idx]*100:.1f}%)")

Sauvegarde

In [ ]:
if distances is not None:
    pd.DataFrame(distances, index=cv_noms, columns=offre_noms).to_csv("distances_tfidf.csv")
    print("\n distances_tfidf.csv sauvegardé")